# TensorFlow Learning (Python)

In [1]:
import numpy as np
import tensorflow as tf

print(tf.version.VERSION)
print(tf.version.COMPILER_VERSION)
print(tf.version.GRAPH_DEF_VERSION)

1.13.1
MSVC 190024215
27


# Tensor
A tensor is a generalization of vectors and matrices to potentially higher dimensions. Internally, TensorFlow represents tensors as n-dimensional arrays of base datatypes.

Each element in the Tensor has the same data type, and the data type is always known.

The rank of a tf.Tensor object is its number of dimensions.

#### Common Tensors
tf.Variable 

tf.constant

tf.placeholder

tf.SparseTensor 

#### dtype
QUANTIZED_DTYPES,  bfloat16,  bool,  complex128,  complex64,  double,  float16,  float32,  float64,  half,  int16,  int32,  int64,  int8,  qint16,  qint32,  qint8,  quint16,  quint8,  resource,  string,  uint16,  uint32,  uint64,  uint8,  variant

In [2]:
rank_zero = tf.Variable(3, name='rank0_tensor')
rank_one = tf.Variable([1,2999,3], name='rank1_tensor')
rank_two = tf.Variable([[2,34,4],[5,67,7],[8,910,10]])
print(tf.rank(rank_zero))
print(tf.rank(rank_one))

element2 = rank_one[1]

print(element2)

with tf.Session() as sess:
    initVars = tf.global_variables_initializer()
    sess.run(initVars)
    print(sess.run(element2))
    print('Rank - ', sess.run(tf.rank(rank_zero)))
    print('Rank - ', sess.run(tf.rank(rank_one)))
    print('Rank - ', sess.run(tf.rank(rank_two)))

Instructions for updating:
Colocations handled automatically by placer.
Tensor("Rank:0", shape=(), dtype=int32)
Tensor("Rank_1:0", shape=(), dtype=int32)
Tensor("strided_slice:0", shape=(), dtype=int32)
2999
Rank -  0
Rank -  1
Rank -  2


# Basic Structure of Tensor Flow

You might think of TensorFlow Core programs as consisting of two discrete sections:

1. Building the computational graph (a tf.Graph).
2. Running the computational graph (using a tf.Session).

A computational graph is a series of TensorFlow operations arranged into a graph. The graph is composed of two types of objects.

1. tf.Operation (or "ops"): The nodes of the graph. Operations describe calculations that consume and produce tensors.
2. tf.Tensor: The edges in the graph. These represent the values that will flow through the graph. Most TensorFlow functions return tf.Tensors.

## Basic Structure (Code)

#### Construction phase (Create Tensors and Graphs)

In [3]:
a = tf.Variable(3, dtype=tf.float32, name='a')
b = tf.Variable(5, dtype=tf.float32, name='b')

# Y = abs(sin( (a^2 + b^2)^0.5  )   )
Y = tf.abs(tf.sin(tf.pow(tf.add(tf.multiply(a,a), tf.multiply(b,b)), 0.5))) #Create the computation graph
## Y = tf.abs(tf.sin(  (a**2 + b**2)**0.5  )) <----- You can also write it directly

In [4]:
initVars = tf.global_variables_initializer()

#### Execution Phase

In [5]:
sess = tf.Session() #Create a TF session
sess.run(initVars) #initlize variables
print(sess.run(Y)) #Execute the computation graph
sess.close() #Clsoe the session

0.4369757


## Gradient Descent Optimizer

In [6]:
x = tf.Variable(10, dtype=tf.float32, name='x')
three = tf.constant(3, dtype=tf.float32, name='three')
Y = (x - three)**2

#Create the GD optimizer object
GD_Optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.5)
training_op = GD_Optimizer.minimize(Y)

initVars = tf.global_variables_initializer()
sess = tf.Session()
sess.run(initVars)

for i in range(100): #Run 100 iterations of Gradient Descent
    sess.run(training_op) #One Step of gradient descent

print('x = ',sess.run(x))
sess.close()

x =  3.0


In [7]:
a = tf.Variable(10, dtype=tf.float32, name='x')
b = tf.Variable(180, dtype=tf.float32, name='y')
three = tf.constant(3, dtype=tf.float32, name='three')
Y = (a - three)**2 + (b - 50)**2

#Create the GD optimizer object
GD_Optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.5)
training_op = GD_Optimizer.minimize(Y)

initVars = tf.global_variables_initializer()
sess = tf.Session()
sess.run(initVars)

for i in range(100): #Run 100 iterations of Gradient Descent
    sess.run(training_op) #One Step of gradient descent

print('a = ',sess.run(a))
print('b = ',sess.run(b))
sess.close()

a =  3.0
b =  50.0


# tf.random

In [8]:
normal_dist = tf.random.normal(shape=(1,10), mean=1, stddev=5, name='normal')
uniform_dist = tf.random.uniform(shape=(2,2), minval=9, maxval=15, dtype=tf.int32, seed=1, name='Uniform')

with tf.Session() as sess:
    print(sess.run(normal_dist))
    print(sess.run(uniform_dist))

[[ 1.6418753 -4.429711  -6.9788074  3.6827345 -2.686794  -6.782597
   9.840202  -2.203504  -6.306733   8.909157 ]]
[[12 14]
 [12 13]]


# Saving and Restoring Models

In [9]:
a = tf.Variable(1000, dtype=tf.float32, name='x')
b = tf.Variable(180, dtype=tf.float32, name='y')
Y = (a - 699)**2 + (b - 50)**2

# Create a saver -----------
saver = tf.train.Saver()

In [10]:
#Create the GD optimizer object
GD_Optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.01)
training_op = GD_Optimizer.minimize(Y)

initVars = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(initVars)
    for i in range(20): #Run 100 iterations of Gradient Descent
        sess.run(training_op) #One Step of gradient descent
        saver.save(sess, './MyModel.ckpt')

    print('a = ',sess.run(a))
    print('b = ',sess.run(b))


a =  899.9501
b =  136.78902


In [11]:
with tf.Session() as RestoredSess:
    saver.restore(RestoredSess, './MyModel.ckpt')
    print(RestoredSess.run(a))

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./MyModel.ckpt
899.9501
